In [16]:
from typing import List, Dict, Tuple
import numpy as np

from lib.helpers import prod

In [19]:
class tarray:
    def __init__(self, shape, dtype, offset=0, order='C', strides=None, buffer=None,):
        """
        Initialize an array.

        Parameters:
        shape (tuple of ints): The shape of the array.
        dtype (data-type, optional): The data type of the array elements.
        buffer (buffer-like, optional): Object exposing buffer interface.
        offset (int, optional): Offset of array data in buffer.
        strides (tuple of ints, optional): Strides of data in memory.
        order ({'C', 'F'}, optional): Row-major (C-style) or column-major (Fortran-style) order.
        """

        self._shape = shape
        self.dtype = dtype
    
        self.itemsize = self._get_itemsize(dtype) # bytes per element

        if buffer and len(buffer) != self.size: raise Exception("Data size does not match shape")
        self.data = buffer if buffer else self._allocate_buffer(self.size, self.itemsize)
        self.base = None if not buffer else buffer

        self.nbytes = self.itemsize * self.size # total bytes
        self.offset = offset # offset in bytes
        self.order = order # order of the array (C or F)
        self.strides = strides if strides else self._cstrides(shape, self.itemsize) # strides in bytes

        self.isContiguous = self._check_contiguous(buffer, shape, strides, self.itemsize) 
        self._validate()

    # ! Methods called by the constructor ======================================
    def _allocate_buffer(self, size, itemsize):
        """Allocates a buffer of the given shape and dtype"""
        return bytearray(size * itemsize)
    
    def _get_itemsize(self, dtype):
        map = {"fp32": 4}
        if dtype in map: return map[dtype]
        raise NotImplementedError(f"Type {dtype} is not supported yet")

    def _check_contiguous(self, buffer, shape, strides, itemsize):
        """Checks if the array is contiguous"""
        if not buffer or not strides or len(shape)==1: return True # if there's no buffer or strides, it's contiguous, or if it's 1D
        # TODO will need to change this method to _cstridesbytes if we're storing bytes instead of elements
        return self._cstrides(shape, itemsize) == strides # return if the strides are the same as the expected strides

    def _validate(self):
        """Validate the arguments passed to the constructor"""
        if self.order != 'C': raise NotImplementedError("Only C order is supported right now") # check ourder
        if len(self.shape) != len(self.strides): raise ValueError("Shape and strides must have same length") # check shape and strides
        if not isinstance(self.offset, int): raise TypeError("Offset must be an integer") # check offset
        if not isinstance(self.shape, tuple): raise TypeError("Shape must be a tuple") # check shape
        if not isinstance(self.strides, tuple): raise TypeError("Strides must be a tuple") # check strides
        if not isinstance(self.data, bytearray): raise TypeError("Data must be a bytearray")
        if not isinstance(self.order, str): raise TypeError("Order must be a string")
        # TODO check valid dtype goes here
        # TODO check strides match the buffer (in bounds)

    # ! Internal methods =======================================================    
    def _cstrides(self, shape: Tuple[int, ...], itemsize: int):
        """
        Returns a tuple of strides for a C-ordered array (in bytes)
        $$n^X = \sum^{N-1}_{i=1}n_i s^x_i$$ where $s^x_i$ gives the *stride* for dimension *i*.
        $$s^C_i = \prod^{N-1}_{j=i+1}d_j=d_{i+1}d_{i+2}\cdots d_{N-1},$$
        """
        return tuple([itemsize*prod(shape[i+1:]) for i in range(len(shape))])
    
    def _fstrides(self, shape: Tuple[int, ...], itemsize: int):
        """
        Returns a tuple of strides for a F-ordered array (in bytes)
        $$n^X = \sum^{N-1}_{i=1}n_i s^x_i$$ where $s^x_i$ gives the *stride* for dimension *i*.
        $$s^F_i = \prod^{i-1}_{j=0}d_j=d_0d_1\cdots d_{i-1}$$
        """
        return tuple([itemsize*prod(shape[0:i]) for i in range(len(shape))]) # TODO check this (slicing is exclusive so we dont need to subtract 1 - i think)

    # ! Utility methods ========================================================

    # ! Public methods =========================================================

    # ! Indexing methods =======================================================
    # TODO: see page 79 for a list of all indexing methods
    def __getitem__(self, key):
        """
        Get an item or slice from the array.
        """
        # ? ARRAY INDEXING NOTES (pg. 79-84) ================================================
        # - two types, basic slicing and advanced indexing
        # - triggered depending on obj
        # - Basic Slicing
        #     - pythons basic slicing to N dimension
        #     - occurs when obj is a slice object `[start:stop:step]`
        #     - also works with a list of slice objects, ..., newaxis
        #     - basic slice syntax is `i:j:k` where `i` is the starting index `j` is the stopping index and `k` is the step $k\neq 0$ This selects the `m` elements (in the corresponding dimension) with index values $i,i+k,\ldots,i+(m-1)K<j$. 
        #     - assume $n$ is the number of elements in the dimension being sliced. Then if $i$ is not given it defaults to 0 for $k>0$ and $n$ for $k<0$. If $j$ is not given it dfefaults to $n$ for $k>0$ and -1 for $k<0$. If $k$ is not fiven it defaults to 1. '::' is the same as ':' and means select all indices on that axis
        #     - If the number of object in the selection tuple is less than $N$, then ': is assumed for any remaing dimensions
        #     - Ellipsis expand to the number of ':' objects needed to make the selection tuple. only one ellipses is expanded
        #     ...
        # - Advanced Selection
        #     - triggered when the selecton object, obj, is a *non tuple sequence object, a tarray, or a tuple with one sequence object or tarray*. Two types of indexing: integer and Boolearn. Advanced selection always returns a copy (instead of a view)
        # ? =====================================================================

        # Case 1: key is an integer
        if isinstance(key, int):
            if key >= self.size: raise IndexError("Index out of bounds")
            return self.data[self.offset + key*self.itemsize]
        
        # Case 2: key is a slice
        if isinstance(key, slice):
            start = key.start if key.start else 0
            stop = key.stop if key.stop else self.size
            step = key.step if key.step else 1
            # TODO This is what copilot autocompleted, but im thinking we might have to recalculate the strides and offset
            return tarray(shape=(stop-start,), dtype=self.dtype, buffer=self.data, offset=self.offset+start*self.itemsize, strides=(step*self.itemsize,), order=self.order) 
        
        # Case 3: key is a tuple
        if isinstance(key, tuple):
            # TODO check if this is the right way to do this
            return self.__getitem__(key[0]).__getitem__(key[1:])
        
        # Case 4: key is a list
        if isinstance(key, list): return self.__getitem__(tuple(key))

        # Case 5: key is a tarray
        if isinstance(key, tarray): raise NotImplementedError("tarray indexing is not supported yet")

        # Case 6: key is something else
        raise TypeError(f"Invalid key type {type(key)}")


    def __setitem__(self, key, value):
        """
        Set an item or slice in the array.
        """

    # ! Shape manipulation methods ==============================================
    def reshape(self, new_shape):
        """
        Returns an array containing the same data with a new shape.

        Parameters:
        new_shape (tuple of ints): The new shape of the array.
        """

        # ? If possible the new array will *reference* the data of the old one. If the data needs to be moved then the new array will contain a copy of the data.

        if not isinstance(new_shape, tuple): raise TypeError("Shape must be a tuple") # check shape is a tuple
        if not all(isinstance(x, int) for x in new_shape): raise TypeError("Shape must be a tuple of ints") # make sure all elements are ints
        if prod(new_shape) != self.size: raise ValueError("Total size of new array must be unchanged") # check total size is unchanged
        if not all(x > 0 for x in new_shape): raise ValueError("Shape must be positive") # check all elements are positive and nonzero

        # Case 1: same shape as before
        if new_shape == self.shape: return self # TODO should this return self (like it is now) or a view (remove this check and let case 2 handle it)

        # Case 2: simple reshape with no permutation
        if len(new_shape) == len(self.shape): # TODO is this the best check?
            # calculate new strides
            # return new array (view). 
            pass

    def transpose(self, *axes):
        """
        Returns a view of the array with axes transposed.

        Parameters:
        axes (sequence of ints, optional): By default, reverse the dimensions.
        """
        # if self.ndim < 2: returns view of self

    # ? Mathematical operations =================================================
    # ! These will all be overloaded operators ==================================
    # numpy does this by defining ufuncs (universal functions) which are functions that operate on ndarrays
    # Basis ops
        # - add, sub, mul, div, truediv, floordiv, mod, divmod, pow, lshift, rshift, and, or, xor
        # - implemented element by element for same shape or broadcastable
    # In place
        # - iadd, isub, imul, idiv, itruediv, ifloordiv, imod, ipow, ilshift, irshift, iand, ixor, ior
    # Unary
        # - neg, pos, abs, invert
        # - via ufuncs except for pos
        # - complex, int, long, float, odc, hex
    # ? ========================================================================
    
    # TODO: see page 164 for a list of all ops as Ufuncs
        
    # ! Basic ops ==============================================================
    
    # ! inplace ops ============================================================
    # TODO: See page 77
        
    # ! Unary ops ==============================================================
    # TODO: See page 78
        
    # ! Reduction ops ==========================================================
    # TODO: See page 68
        
    # ! Linear algebra ops =====================================================
    # TODO: see page 90
        
    # ! Comparison methods ======================================================

    # ! Special methods ========================================================
    # TODO: See page 71 for a list of all special methods
    def __len__(self): return self.shape[0] # this is what numpy does
    def __nonzero__(self):
        if self.size > 1: raise ValueError("The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()")
        return True if self.size == 1 else False 

    # ! Conversion methods =====================================================
    def to_numpy(self):
        """
        Returns a numpy array with the same data.
        """
        return np.frombuffer(self.data, dtype=self.dtype).reshape(self.shape)

    # ! Properties =============================================================
    @property
    def shape(self):
        """
        Tuple of array dimensions.
        """
        return self._shape

    @property
    def size(self):
        """
        Number of elements in the array.
        """
        return prod(self.shape)

    @property
    def ndim(self):
        """
        Number of array dimensions.
        """
        return len(self.shape)
    
    @property
    def T(self):
        """
        Transpose of the array.
        """
        return self.transpose()

    def __str__(self):
        """
        Informal string representation of the array.

        - last axis is printed l to r
        - next-to last is printed top to bottom
        - remaining axes are printed top to bottom with separators

        """
        return f"tarray(shape={self.shape}, dtype={self.dtype}, strides={self.strides}, order={self.order})"

    def __repr__(self):
        """
        String representation of the array.
        """
        return self.__str__()
    
    
def array(object, dtype, copy: bool=False, order: str='C'): # TODO: see page 85
    """
    Creates an array.

    Parameters:
    object (array_like): An array, any object exposing the array interface, an object whose __array__ method returns an array, or any (nested) sequence.
    dtype (data-type, optional): The desired data-type for the array. If not given, then the type will be determined as the minimum type required to hold the objects in the sequence.
    copy (bool, optional): If true (default), then the object is copied. Otherwise, a copy will only be made if __array__ returns a copy, if obj is a nested sequence, or if a copy is needed to satisfy any of the other requirements (dtype, order, etc.).
    order ({'C', 'F'}, optional): Specify the memory layout of the array. If object is not an array, the newly created array will be in C order (row major) unless 'F' is specified, in which case it will be in Fortran order (column major).

    Returns:
    out (tarray): An array object satisfying the specified requirements.
    """
    return tarray(shape=object.shape, dtype=dtype, buffer=object.data, order=order)

In [6]:
f"${(1800000 * (1.0807)**24):.02f}"

'$11593008.13'